### Preparativos do Dataset (remoção de colunas sem variação de valores e categorizando as colunas ordenadas e não ordenadas)

In [1]:
import pandas as pd

BASE = "../datasets/HR-Employee-Attrition.csv"

df = pd.read_csv(BASE)


# Limpando a base de colunas com valores não variam e removendo a
# coluna de identificação (EmployeeNumber)

df = df.drop("EmployeeNumber", axis=1)

for c in df.columns:
    column = df[c]
    if (column.nunique() <= 1):
        print(f"Removendo coluna {c}") 
        df = df.drop(c, axis=1)


# Separando nossas colunas por variáveis categóricas ordenadas e 
# não ordernadas


# Colunas que são categóricas e não ordenadas
columns_categorical_not_ordered = [
    'BusinessTravel',
    'Department',
    'EducationField',
    'Gender',
    'JobRole',
    'MaritalStatus',
    'OverTime'
]

# Colunas que são categóricas e ordernadas
columns_categorical_ordered = [
    'Education',
    'EnvironmentSatisfaction',
    'JobSatisfaction',
    'JobInvolvement',
    'JobLevel',
    'PerformanceRating',
    'RelationshipSatisfaction',
    'StockOptionLevel',
    'WorkLifeBalance',
    'JobInvolvement'
]

target_column = ['Attrition']


numeric_columns = [
    column
    for column in df.columns
    if column not in (columns_categorical_ordered + columns_categorical_not_ordered + target_column)
]


# Convertendo nossas colunas não ordernadas para o tipo category

for column in columns_categorical_not_ordered:
    df[column] = df[column].astype("category")

Removendo coluna EmployeeCount
Removendo coluna Over18
Removendo coluna StandardHours


### Análise da coluna salário

Vamos considerar a coluna "MonthlyIncome" como a coluna salário, podemos verificar a média salarial por departamento:

In [13]:
df.groupby("Department", observed=True)["MonthlyIncome"].mean()

Department
Human Resources           6654.507937
Research & Development    6281.252862
Sales                     6959.172646
Name: MonthlyIncome, dtype: float64

Podemos ver um certo padrão se fizermos um agrupamento com a coluna "Attrition"

In [16]:
df.groupby(["Department", "Attrition"], observed=True)["MonthlyIncome"].mean()

Department              Attrition
Human Resources         No           7345.980392
                        Yes          3715.750000
Research & Development  No           6630.326087
                        Yes          4108.075188
Sales                   No           7232.240113
                        Yes          5908.456522
Name: MonthlyIncome, dtype: float64

É possível concluir com base nos dados acima de que a média salarial de funcionários que saíram da empresa é menor do que a de funcionários que permaneceram.

#### Reproduzindo o cenário acima utilizando o pivot_table

In [30]:
df.pivot_table(index="Department", columns="Attrition", values="MonthlyIncome")

Attrition,No,Yes
Department,,
Human Resources,7345.980392,3715.750000
Research & Development,6630.326087,4108.075188
Sales,7232.240113,5908.456522


#### Visualizando a média salarial por cargo

In [ ]:
df.groupby("JobRole", observed=True)["MonthlyIncome"].mean()

#### Visualizando a média salarial por cargo + nível do cargo

In [43]:
df.groupby(["JobRole", "JobLevel"], observed=True)["MonthlyIncome"].mean()

JobRole                    JobLevel
Healthcare Representative  2            5865.346154
                           3            9363.954545
                           4           12973.000000
Human Resources            1            2733.212121
                           2            5563.461538
                           3            9623.000000
Laboratory Technician      1            2854.590000
                           2            4455.625000
                           3            5998.000000
Manager                    3           12233.000000
                           4           16613.276596
                           5           19183.976744
Manufacturing Director     2            5612.755556
                           3            9307.044444
                           4           13383.000000
Research Director          3           12721.571429
                           4           16429.038462
                           5           19204.807692
Research Scientist         1

#### Convertendo o retorno do groupby para um DataFrame utilizando o .to_frame()

In [52]:
df_mean_by_job_role_and_level = df.groupby(["JobRole", "JobLevel"], observed=True)["MonthlyIncome"].mean()
df_mean_by_job_role_and_level = df_mean_by_job_role_and_level.to_frame()
df_mean_by_job_role_and_level

MonthlyIncome
JobRole                   JobLevel               
Healthcare Representative 2           5865.346154
                          3           9363.954545
                          4          12973.000000
Human Resources           1           2733.212121
                          2           5563.461538
                          3           9623.000000
Laboratory Technician     1           2854.590000
                          2           4455.625000
                          3           5998.000000
Manager                   3          12233.000000
                          4          16613.276596
                          5          19183.976744
Manufacturing Director    2           5612.755556
                          3           9307.044444
                          4          13383.000000
Research Director         3          12721.571429
                          4          16429.038462
                          5          19204.807692
Research Scientist        1           2827.649573
                          2           4818.912281
                          3           9724.000000
Sales Executive           2           5800.884120
                          3           9124.987342
                          4          13202.500000
Sales Representative      1           2506.723684
                          2           3921.000000

Para facilitar o entendimento de forma visual de maiores e menores salários podemos utilizar o style

In [55]:
df_mean_by_job_role_and_level.style.background_gradient(cmap="RdYlGn", axis="index")

É interessante analisar que o cargo "Sales Representative" tem o menor salário (o teto é em média 3921) e é também a área onde o "Attrtion" é maior conforme vimos com o crosstab

In [57]:
pd.crosstab(df["Department"], df["Attrition"], normalize="index")

Attrition,No,Yes
Department,,
Human Resources,0.809524,0.190476
Research & Development,0.861602,0.138398
Sales,0.793722,0.206278


Proporcionalmente falando a maior média de "Attrition" é para a área de vendas, são 21%.